(melodic-pattern-discovery)=
# Melodic pattern dicovery

Intro here...

In [ ]:
%pip install compiam

## Importing compiam to the project
import compiam

# Import extras and supress warnings to keep the tutorial clean
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

from compiam import load_model
from compiam.visualisation.waveform_player import Player
from compiam.melody.pattern.sancara_search.extraction.evaluation import load_annotations_brindha, to_aeneas

In [ ]:
audio_path = os.path.join("../", "audio", "sharanu_janakana.mp3")
title = 'Sharanu Janakana'

In [ ]:
Player(title, audio_path)